# ANLI Classification using DeBERTa (GPU ENABLED)
**Goals**
- End-to-end ML pipeline: EDA → Preprocessing → Training → Evaluation → Save & Deploy.
- Save metrics, plots, confusion matrix, model, and tokenizer.
- Provide GitHub & Docker deployment instructions.


In [ ]:
# Cell: Imports & Utilities
import os
import random
import json
from pathlib import Path
from datetime import datetime
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score, f1_score,
    precision_recall_fscore_support, confusion_matrix,
    classification_report,

import torch
from datasets import load_dataset, concatenate_datasets, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    EarlyStoppingCallback, set_seed


# Output directories
OUT_DIR = Path("/content/anli_best_results_deberta_r2")
OUT_DIR.mkdir(parents=True, exist_ok=True)

def save_json(obj, path):
    with open(path, "w") as f:
        json.dump(obj, f, indent=4, default=str)

# Set seed for reproducibility
set_seed(42)

# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


# 0. CONFIGURATION

In [ ]:
USE_ALL_ROUNDS = False 
MODEL_NAME = "microsoft/deberta-v3-large"
MAX_LENGTH = 384  # increased from 256 -> 384
BATCH_SIZE = 8  # per device (kept same as your original)
GRAD_ACC = 4  # gradient accumulation
LR = 1e-5
NUM_EPOCHS = 3
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {DEVICE}")


# 1. REPRODUCIBILITY

In [ ]:
set_seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
if DEVICE == "cuda":
    torch.cuda.manual_seed_all(SEED)


# 2. LOAD DATA & SIMPLE EDA

In [ ]:
dataset = load_dataset("facebook/anli")

train = dataset["train_r2"]
dev   = dataset["dev_r2"]
test  = dataset["test_r2"]

label_map = {0: "entailment", 1: "neutral", 2: "contradiction"}

# Quick EDA function
def dataset_stats(ds, name):
    labels = np.array(ds["label"])
    unique, counts = np.unique(labels, return_counts=True)
    print(f"{name} size: {len(ds)}")
    for u, c in zip(unique, counts):
        print(f"  {label_map[int(u)]:12s}: {c} ({c/len(ds):.2%})")

dataset_stats(train, "Train")
dataset_stats(dev, "Dev")
dataset_stats(test, "Test")

# Save a quick EDA file
unique, counts = np.unique(np.array(train["label"]), return_counts=True)
train_class_counts = {int(u): int(c) for u, c in zip(unique, counts)}

eda_summary = {"train_size": len(train), "dev_size": len(dev), "test_size": len(test),"train_class_counts": train_class_counts}

save_json(eda_summary, OUT_DIR / "eda_summary.json")
print(f"Saved EDA summary to {OUT_DIR / 'eda_summary.json'}")


In [ ]:
# BALANCE THE TRAINING SET (UPSAMPLING MINORITY CLASSES)

print("Original class counts:", Counter(train["label"]))
label_counts = Counter(train["label"])
max_count = max(label_counts.values())

balanced_splits = []
for label in [0, 1, 2]:
    subset = train.filter(lambda x: x["label"] == int(label))
    subset_dict = subset.to_dict()
    n = len(subset)
    if n == 0:
        continue
    # How many repeats to exceed or reach max_count
    repeat_factor = max_count // n + (1 if max_count % n != 0 else 0)
    # Repeat each field in subset_dict
    repeated = {k: (v * repeat_factor)[: max_count] for k, v in subset_dict.items()}
    upsampled = Dataset.from_dict(repeated)
    upsampled = upsampled.shuffle(seed=SEED)
    balanced_splits.append(upsampled)

# Concatenate balanced splits and shuffle
train_balanced = concatenate_datasets(balanced_splits).shuffle(seed=SEED)
print("Balanced class counts:", Counter(train_balanced["label"]))
print("Balanced train size:", len(train_balanced))

# Replace train with balanced version for the rest of pipeline
train = train_balanced

# Save new EDA after balancing
unique, counts = np.unique(np.array(train["label"]), return_counts=True)
train_class_counts_balanced = {int(u): int(c) for u, c in zip(unique, counts)}
eda_summary["train_class_counts_balanced"] = train_class_counts_balanced
eda_summary["train_balanced_size"] = len(train)
save_json(eda_summary, OUT_DIR / "eda_summary.json")


# 3. TOKENIZER & PREPROCESSING

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess(batch):
    return tokenizer(batch["premise"], batch["hypothesis"],
                     truncation=True, max_length=MAX_LENGTH, padding=False)

train_tok = train.map(preprocess, batched=True, remove_columns=train.column_names)
dev_tok   = dev.map(preprocess,   batched=True, remove_columns=dev.column_names)
test_tok  = test.map(preprocess,  batched=True, remove_columns=test.column_names)

# add labels back and set to torch format
train_tok = train_tok.add_column("labels", train["label"])
dev_tok   = dev_tok.add_column("labels", dev["label"])
test_tok  = test_tok.add_column("labels", test["label"])

train_tok.set_format(type="torch")
dev_tok.set_format(type="torch")
test_tok.set_format(type="torch")

collator = DataCollatorWithPadding(tokenizer)
print("Tokenization complete. Example tokenized fields:", list(train_tok.features.keys())[:10])


# 4. MODEL

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
model.to(DEVICE)
print(f"Loaded model {MODEL_NAME} and moved to {DEVICE}")


# 5. METRICS & EVAL FUNCTIONS

In [ ]:
def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)

    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, labels=[0,1,2], average=None, zero_division=0)
    macro = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "accuracy": float(acc),
        "macro_f1": float(macro),
        "f1_entailment": float(f1[0]),
        "f1_neutral": float(f1[1]),
        "f1_contradiction": float(f1[2]),
    }

def get_history(trainer):
    history = trainer.state.log_history
    return history


# 6. TRAINING ARGUMENTS & CALLBACKS

In [ ]:
training_args = TrainingArguments(
    output_dir=str(OUT_DIR),
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=GRAD_ACC,
    warmup_ratio=0.1,
    weight_decay=0.1,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="steps",
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="macro_f1",
    greater_is_better=True,
    fp16=True if DEVICE == "cuda" else False,
    report_to="none",
)

earlystop = EarlyStoppingCallback(early_stopping_patience=3)
print("TrainingArguments prepared.")


# 7. TRAINER SETUP

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=dev_tok,
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
    callbacks=[earlystop],
)
print("Trainer initialized.")


# 8. TRAIN

In [ ]:
train_start = datetime.utcnow().isoformat()
train_result = trainer.train()
train_end = datetime.utcnow().isoformat()

history = get_history(trainer)
save_json({"train_start": train_start, "train_end": train_end, "history": history}, OUT_DIR / "trainer_history.json")

trainer.save_model(str(OUT_DIR / "best_model"))
tokenizer.save_pretrained(str(OUT_DIR / "best_model"))
print(f"Training completed. Model and tokenizer saved to {OUT_DIR / 'best_model'}")


# 9. EVALUATION & METRICS (DEV & TEST)

In [ ]:
dev_metrics = trainer.evaluate(eval_dataset=dev_tok)
test_metrics = trainer.evaluate(eval_dataset=test_tok)

print("DEV:", dev_metrics)
print("TEST:", test_metrics)

save_json({"dev": dev_metrics, "test": test_metrics}, OUT_DIR / "metrics.json")
print(f"Saved dev/test metrics to {OUT_DIR / 'metrics.json'}")


# 10. DETAILED TEST REPORT & CONFUSION MATRIX

In [ ]:
pred_output = trainer.predict(test_tok)
logits = pred_output.predictions
preds = np.argmax(logits, axis=1)
labels = pred_output.label_ids

# Classification Report
clf_report = classification_report(labels, preds, target_names=[label_map[i] for i in [0,1,2]], zero_division=0, output_dict=True)
save_json(clf_report, OUT_DIR / "classification_report.json")
print("Classification report saved.")

# Confusion matrix & plot
cm = confusion_matrix(labels, preds, labels=[0,1,2])
cm_display = {
    "matrix": cm.tolist(),
    "labels": [label_map[i] for i in [0,1,2]],
}
save_json(cm_display, OUT_DIR / "confusion_matrix.json")

# Plot confusion matrix
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=[label_map[i] for i in [0,1,2]], yticklabels=[label_map[i] for i in [0,1,2]])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - ANLI R2 Test")
plt.tight_layout()
plt.savefig(OUT_DIR / "confusion_matrix.png", dpi=200)
plt.show()
print(f"Saved confusion matrix image to {OUT_DIR / 'confusion_matrix.png'}")


# 11. TRAINING CURVES (LOSS / METRICS) PLOT

In [ ]:
history = trainer.state.log_history

steps = [h.get("step") for h in history if "step" in h and "loss" in h]
train_losses = [h["loss"] for h in history if "loss" in h]

eval_steps = [h["step"] for h in history if "step" in h and any(k.startswith("eval_") for k in h)]
eval_f1 = [h.get("eval_macro_f1") for h in history if "eval_macro_f1" in h]
eval_acc = [h.get("eval_accuracy") for h in history if "eval_accuracy" in h]
eval_loss = [h.get("eval_loss") for h in history if "eval_loss" in h]

# Plot training loss
if len(steps) and len(train_losses):
    plt.figure(figsize=(7,5))
    plt.plot(steps[:len(train_losses)], train_losses, marker='o')
    plt.xlabel("Steps")
    plt.ylabel("Training Loss")
    plt.title("Training Loss Curve")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "train_loss.png", dpi=200)
    plt.show()
    print(f"Saved training loss to {OUT_DIR / 'train_loss.png'}")

# Plot eval metrics
if len(eval_steps):
    plt.figure(figsize=(7,5))
    if any(val is not None for val in eval_f1):
        plt.plot(eval_steps, eval_f1, label="Eval Macro F1", marker='o')
    if any(val is not None for val in eval_acc):
        plt.plot(eval_steps, eval_acc, label="Eval Accuracy", marker='o')
    if any(val is not None for val in eval_loss):
        plt.plot(eval_steps, eval_loss, label="Eval Loss", marker='o')
    plt.legend()
    plt.xlabel("Steps")
    plt.title("Evaluation Metrics Curve")
    plt.tight_layout()
    plt.savefig(OUT_DIR / "eval_metrics.png", dpi=200)
    plt.show()
    print(f"Saved eval metrics to {OUT_DIR / 'eval_metrics.png'}")


# 12. SAVE PARAMETERS & REPRODUCIBILITY NOTES

In [ ]:
repro = {
    "model_name": MODEL_NAME,
    "max_length": MAX_LENGTH,
    "batch_size": BATCH_SIZE,
    "grad_acc": GRAD_ACC,
    "learning_rate": LR,
    "epochs": NUM_EPOCHS,
    "seed": SEED,
    "use_all_rounds": USE_ALL_ROUNDS,
    "device": DEVICE,
    "notes": "Check trainer_history.json for stepwise logs."
}
save_json(repro, OUT_DIR / "reproducibility.json")
print(f"Saved artifacts to {OUT_DIR.resolve()}")
